# AI Agents 101

In this notebook we will explore AI Agents using Langchain

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

# Installations
- Langchain
- Langchain-groq
- python-dotenv

In [1]:
! pip install langchain

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 12.4 MB/s eta 0:00:00
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached typing_inspection-0.4.2-py3-none-any.whl (14 kB)

   -- -------------------------------------  1/18 [xxhash]
   ------ ---------------------------------  3/18 [tenacity]
   ------ ---------------------------------  3/18 [tenacity]
   -------- ------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install langchain-groq

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)

   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ---------------------------------------- 3/3 [langchain-groq]




[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Setting up Environment Variables

In [2]:
import os #Importing os to access the file
from dotenv import load_dotenv #importing dotenv to load the .env file

load_dotenv(dotenv_path = ".env")
api_key = os.getenv("GROQ_API_KEY")

In [3]:
#langchain import to create agents
from langchain.agents import create_agent

In [6]:
#Define a tool for the agent to call
def get_weather(city: str) -> str:
    """Get weather for a given city"""
    return f"It's always sunny in {city}!"


In [13]:
#langchain import to define our chat model
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "qwen/qwen3-32b", #model
    model_provider="groq", #model provider
    temperature = 0 #temperature
)

### Mini Documentation
- When you wrap a model with init_chat_model, you freeze these settings.
 This ensures your agent behaves the same way every time.

In [49]:
#Create an agent with an LLM model along with the tools and system prompt
agent = create_agent(
    model = model,
    tools = [get_weather],
    system_prompt="""You are an weather analyst, you are responsible to identify if an given text is an question relevant to seeking information about weather. Always extract the city from a given query and pass it to the tool, fetch the response and generate the final response.
    """
)

In [50]:
#Viewing the results
agent.invoke(
    {
        "messages": [{"role":"user", "content": "What's the weather in Chennai"}]
    }
)

{'messages': [HumanMessage(content="What's the weather in Chennai", additional_kwargs={}, response_metadata={}, id='3c57d0de-7bd2-47a4-b765-cce99bf48677'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking, "What\'s the weather in Chennai?" I need to determine if this is a weather-related question. The query clearly mentions "weather" and specifies the city "Chennai." My task is to extract the city from the query and use the provided tool to get the weather information.\n\nFirst, I\'ll check if the function get_weather is available. Yes, it is. The parameters required are the city, which in this case is Chennai. There\'s no ambiguity here since the user directly mentioned the city. I don\'t need to ask for clarification. \n\nNext, I should structure the tool call correctly. The function name is get_weather, and the argument should be a JSON object with the city key. So the tool call would be {"name": "get_weather", "arguments": {"city": "Chennai"

In [51]:
#Storing the results
result = agent.invoke(
    {
        "messages": [{"role":"user", "content":"What's the weather in Chennai"}]
    }
)

In [52]:
#Printing only the end results
print(result.get("messages")[-1].content)

The weather in Chennai is currently sunny! It seems like a pleasant day there. 😊


### Predictable AI Agent
- Predictable AI Agents are AI agents that do not guess anything.
 - If it needs information, it must call the correct tool.
 - This makes the system predictable and safe.

### Mini Documentation
- We use tool to define a function as tool in langchain, so the langchain orchestration understands with function is tool, which isn't.
- ToolRuntime is used modify tool's data access during agent's Runtime.
- Tool Strategy defines how the agent output should be.

In [23]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents.structured_output import ToolStrategy

In [19]:
#We are defining weather tool for our agent

@tool
def get_weather_for_location(city: str) -> str:
    """
    A simple tool that refers weather information.
    In a real system, this could call a weather API.
    """
    return f"It's summy in {city}"

In [7]:
"""
    This class acts like the context to agent, where if user_id is passed in,
    it personalizes for the user, this is useful when the agents are connected
    to databases or APIs.
"""

@dataclass
class Context:
    user_id: str

In [8]:
@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """
    A context-aware tool.
    It reads the user_id from runtime.context and returns the user's locations.

    In real world, you can replace this with a real database or user profile lookup.
    """

    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "Banglore"


### Mini Documentation
- The first tool is straightforward: the agent calls it when it already knows the city.

- The second tool is the interesting part. Instead of taking direct arguments, it receives a runtime object.

In [10]:
#Defining response structure while Langchain maintains it's output structure
@dataclass
class ResponseFormat:
    punny_response: str
    weather_conditions: str | None = None

In [11]:
from langgraph.checkpoint.memory import InMemorySaver

#Simple In-Memory Storage for conversation history
checkpointer = InMemorySaver()

In [12]:
# System Prompt
SYSTEM_PROMPT = """
You are an expert weather forecaster, who speaks in puns. You have access to two tools:
- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location
If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location.
"""

In [24]:
agent = create_agent(
    model = model, #model
    system_prompt=SYSTEM_PROMPT, #System Prompt
    tools=[get_user_location,get_weather_for_location], #Tools defined
    context_schema=Context, #Context we have defined
    response_format=ToolStrategy(ResponseFormat), #As the name suggests,it's context format
    checkpointer=checkpointer #checkpointer for memory
)

In [25]:
# Thread ID groups all messages into a single session.
# Reusing the same ID lets the agent remember context.

config = {"configurable": {"thread_id":"1"}}

In [26]:
"""
Trail Output #1
"""

response = agent.invoke(
    {
        "messages":[{"role":"user", "content":"What is the weather outside ?"}]
    },
    config = config,
    context=Context(user_id="1")
)

In [29]:
print(response['structured_response'])

ResponseFormat(punny_response="It's a 'sun' day in Florida!", weather_conditions='summy')


In [30]:
"""
Trail Output #2
"""

response = agent.invoke(
    {
        "messages":[{"role":"user", "content":"What did I last asked you about ?"}]
    },
    config = config,
    context=Context(user_id="1")
)

In [31]:
print(response["structured_response"])

ResponseFormat(punny_response='Sunny days and forgetful minds! You asked about the weather in Florida.', weather_conditions=None)
